In [1]:
from bs4 import BeautifulSoup as chunky
import requests
import pandas as bear
import regex as re
import random as ran

The HTML varies from webpage to webpage for this particular project so it isn't as clean as I would love for it to 
be but the idea is to scrape discography information for kpop artists and append it to a dictionary

In this step, I went to the discography page for all kpop artists and my intention was to scrape so that i can have a complete list of all the artists names.

In [2]:
link = 'https://kprofiles.com/kpop-discographies/'

In [3]:
step1 = requests.get(link)

In [4]:
soup = chunky(step1.content, 'html.parser')

In [5]:
kpop = soup.find(id='content').get_text()

In [6]:
look = '\n(.+)\sDis.*'

In [7]:
artists = re.findall(look,kpop)

In [306]:
len(artists)

571

In [307]:
artists[:25]

['1014',
 '109',
 '15&',
 '1CHU',
 '1NB',
 '1set',
 '1TEAM',
 '1THE9',
 '2315',
 '24K',
 '2danji',
 '2NE1',
 '2NYNE',
 '2Z',
 '3YE',
 '4CARAT',
 '4Minute',
 '7anhee',
 '7th Sense Unit',
 '82MAJOR',
 '8TURN',
 '9MUSES',
 'AB6IX',
 'ABLUE',
 'ABRY']

From here, I decided to use the random module to generate a random name/group to later explore their discography info. I used the 'artists' list that I just created as my sample pool, which contains 571 kpop artists.

In [285]:
random_pick =ran.choice(artists).lower()

In [286]:
random_pick

'gain'

Some of the artist's names contain symbols and whitespace that would interfere with formatting string in the link (see below). All of the pages follow the same convention: 'https://kprofiles.com/{artist}-discography/, with the only difference being the artists name. If the name has spaces in it, the website has transformed those into hyphens and have completely omitted any special characters. Because of that, I used regex to removed the special characters if found and to change a black space to a hyphen.

In [287]:
sc = r'[!|@|#|$|%|\^|&|\(|\)|\.|:]'
def group(name):
    for i in name:
        new = re.sub(sc,'',name)
        name = re.sub(' ','-', new)
      
        return name

In [288]:
art_name = group(random_pick)

In [289]:
art_name

'gain'

In [290]:
disc_page = f'https://kprofiles.com/{art_name}-discography/'

In [291]:
step2 = requests.get(disc_page)
more_soup = chunky(step2.content, 'html.parser')

In [308]:
disc_soup = more_soup.find('div',class_='herald-section container')

In [309]:
discography = disc_soup.text

Since these pages are made by fans, the bottom of the page contains polls and creator commentary. 
The next few steps works to remove that. If the text contains any of the phrases below, the code will grab the
index of that item and only include text up until that specific index. If there are multiple phrases present in the text
the program will grab the one with the smallest index (comes first) and keep only the text up until that point.

In [294]:
phrase = ["What's your favourite", "Note:","made by","What is your favorite","What's your favorite",f"Which {art_name.upper()}","ResultsPoll","Made by"]

In [295]:
idx = min([words.index(i) for i in phrase if i in words])

In [296]:
idx

1772

In [297]:
nopoll = words[:idx].split('\n')

In [298]:
nopoll

['  Gain DiscographyAdd CommentGain (Brown Eyed Girls) Discography',
 '  Step 2/4',
 'Release Date: October 8, 2010',
 '',
 'Mini Album',
 '1. Nitchell (Baby-G Mix)',
 '2. Irreversible (돌이킬 수 없는) (Title Track)\xa0',
 '3. Tango The Night',
 '4. Esperando',
 '5. Ga In (가인) (歌人)',
 '6. Truth (진실)Bad Temper',
 'Release Date: December 21, 2010',
 '',
 'Single',
 '1. Bad TemperTalk About S',
 'Release Date: October 5, 2012',
 '',
 'Mini Album',
 '1. TinkerBell (팅커벨)',
 '2. Tiredness (그녀를 만나)',
 '3. Bloom (피어나) (Title Track)\xa0',
 '4. The Gaze (시선) (feat. Yoon Jong Shin)',
 '5. Catch Me If You CanNostalgia',
 'Release Date: November 30, 2012',
 '',
 'Single',
 '1. Nostalgia (노스텔지아) (feat Eric of SHINHWA)',
 '2. Nostalgia (노스텔지아) (Inst.)Fxxk U',
 'Release Date: January 28, 2014',
 '',
 'Single',
 '1. Fxxk U (feat. Bumkey)\xa0Truth Or Dare',
 'Release Date: February 6, 2014',
 '',
 'Mini Album',
 '1. Truth or Dare (진실 혹은 대담) (Title Track)',
 '2. Fxxk U (feat. Bumkey)',
 '3. Q&A (feat. Jo Kwon)

The regex usage below will fix some spacing issues in the text

In [299]:
p = r'(\d:\d{2}|\?|[a-z]|\))(\d{1}.|[A-Z])'
q = r'([A-Z][a-z])'

In [300]:
grp = []
for i in nopoll:
    fix = re.sub(p,r'\1 -- \2',i)
    fix2 = re.sub('\xa0','',fix)
    fix3 = re.sub(q,r' \1',fix2)
    fix4 = re.sub('  ',' ',fix3)
    grp.append(fix4.strip())

In [301]:
disc = [i for i in grp if i and 'Discography' not in i]

In [302]:
disc

['Step 2/4',
 'Release Date: October 8, 2010',
 'Mini Album',
 '1. Nitchell ( Baby-G Mix)',
 '2. Irreversible (돌이킬 수 없는) ( Title Track)',
 '3. Tango The Night',
 '4. Esperando',
 '5. Ga In (가인) (歌人)',
 '6. Truth (진실) -- Bad Temper',
 'Release Date: December 21, 2010',
 'Single',
 '1. Bad Temper -- Talk About S',
 'Release Date: October 5, 2012',
 'Mini Album',
 '1. Tinker -- Bell (팅커벨)',
 '2. Tiredness (그녀를 만나)',
 '3. Bloom (피어나) ( Title Track)',
 '4. The Gaze (시선) (feat. Yoon Jong Shin)',
 '5. Catch Me If You Can -- Nostalgia',
 'Release Date: November 30, 2012',
 'Single',
 '1. Nostalgia (노스텔지아) (feat Eric of SHINHWA)',
 '2. Nostalgia (노스텔지아) ( Inst.) -- Fxxk U',
 'Release Date: January 28, 2014',
 'Single',
 '1. Fxxk U (feat. Bumkey) Truth Or Dare',
 'Release Date: February 6, 2014',
 'Mini Album',
 '1. Truth or Dare (진실 혹은 대담) ( Title Track)',
 '2. Fxxk U (feat. Bumkey)',
 '3. Q&A (feat. Jo Kwon)',
 '4. Black & White',
 '5. Disclosure (폭로)',
 '6. A Tempo ( Inst.) (CD Only) -- A Tem

In [107]:
##kpop_dict = {}

In [ ]:
#Append each artist and their discography to a dictionary

In [303]:
kpop_dict['Artist:',random_pick.upper()]= 'DISCOGRAPHY:',disc

In [304]:
kpop_dict

{('Artist:', 'HORI7ON'): ('DISCOGRAPHY:',
  ['DASH',
   'Release Date: March 22, 2023',
   'Single --DASHDASH (Inst.) --Salamat',
   'Release Date: April 5, 2023',
   'Single --Salamat --Salamat (Inst.) --Lovey Dovey',
   'Release Date: May 31, 2023',
   'Digital Single --Lovey Dovey --Lovey Dovey (Inst.) --Friend-SHIP',
   'Release Date: July 24, 2023',
   '1st Album --METEOR (Prod. --Noa of TFN) --SIX7EENBirthday --Mama --How You Feel (ft. --HAEUN of Lapillus) --Death Or Paradise --Take My Hand --Tiger --Hit Me --Odd Eye --Lovey Dovey --SIX7EEN (Inst.) --CD Only:Bonnie & Clyde (ft. --Leo of TFN) --Light Way (ft. --Noa of TFN)틍 (ft. --Jom of new:ID) --Like a fiction (ft. --Macky of new:ID) --Ready To Burn (ft. --Thad of new:ID) --Silhouette (ft. --Wilson of new:ID) --Cost (ft. --Josh. --L of new:ID) --DASHSalamat --DAYTOUR',
   'Release Date: March 2, 2024',
   'Single --LUCKYLUCKY (Inst.)']),
 ('Artist:', 'MYB'): ('DISCOGRAPHY:',
  ['My Oh My',
   'Release date: August 25, 2015',
   